In [1]:
import pandas as pd
import os
import glob
import openpyxl
import re  # Added for regex replacement of underscores

def validate_and_update_excel_structure():
    """
    This script validates the structure of one or more Excel files against a
    schema defined in 'Columns_trace.xlsx' and then saves the files with
    updated worksheet names while preserving all original formatting.

    For each target Excel file found, it performs the following actions:
    1.  Validates that all entities from the trace file have a corresponding
        worksheet (normalizing names by replacing spaces with underscores).
    2.  Validates that all required columns are present in each worksheet.
    3.  Prints a file-by-file report of any missing elements.
    4.  Saves the inspected file, overwriting it, with:
        - Worksheet names updated to use underscores (_) instead of spaces.
        - ID columns fixed (double underscores replaced with single underscores).
        - All styles preserved.
    """
    #  1. Define File Names and Paths 
    trace_filename = "columns_trace.xlsx"
    current_directory = os.getcwd()
    
    print(f"Searching for files in: {current_directory}\n")

    #  2. Load the Schema from 'Columns_trace.xlsx' 
    try:
        trace_df = pd.read_excel(trace_filename, header=None)
    except FileNotFoundError:
        print(f"ERROR: The schema file '{trace_filename}' was not found.")
        print("Please make sure it is in the same folder as the script.")
        return

    # Create a dictionary to hold the required schema
    schema = {}
    for _, row in trace_df.iterrows():
        entity_name = row[0]
        if pd.notna(entity_name):
            columns = row[1:].dropna().tolist()
            schema[entity_name] = columns
    
    print(f"Successfully loaded schema for {len(schema)} entities from '{trace_filename}'.")

    #  3. Find all Target Data Excel Files 
    target_files = [f for f in glob.glob("*.xlsx") if f != trace_filename]
    
    if not target_files:
        print("\nERROR: No target Excel files (.xlsx) found to validate.")
        print("Please add at least one data file to the folder.")
        return
    
    print(f"Found {len(target_files)} target file(s) to validate and update.")

    # Define the prefixes to look for in ID columns
    id_prefixes = [
        "p_", "w_", "ex_", "m_", "m_vol_", "i_", "PO_", "PO_PAG_",
        "PAG_", "VO_", "e_", "ac_", "inst_", "loc_"
    ]

    #  4. Perform Validation and Update for Each File 
    for target_filename in target_files:
        print(f"\n\n{'='*25}\n Inspecting File: {target_filename} \n{'='*25}")
        
        validation_errors_this_file = False
        
        try:
            target_xls = pd.ExcelFile(target_filename)
            original_sheet_names = target_xls.sheet_names
            sheet_name_map = {name.replace(' ', '_'): name for name in original_sheet_names}
            normalized_target_sheets = list(sheet_name_map.keys())

        except Exception as e:
            print(f"  [ERROR] Could not read or process the file. Reason: {e}")
            continue

        #  4a. Check for Missing Worksheets 
        print("\nChecking for missing worksheets...")
        found_sheets_map = {}
        any_sheets_missing = False
        
        for entity_name in schema.keys():
            normalized_schema_name = entity_name.replace(' ', '_')
            
            if normalized_schema_name not in normalized_target_sheets:
                print(f"  [MISSING] Worksheet for entity '{entity_name}' (expected name: '{normalized_schema_name}') was not found.")
                validation_errors_this_file = True
                any_sheets_missing = True
            else:
                found_sheets_map[entity_name] = sheet_name_map[normalized_schema_name]

        if not any_sheets_missing:
             print("  All required worksheets are present.")

        #  4b. Check for Missing Columns in Existing Worksheets 
        print("\nChecking for missing columns in existing worksheets...")
        if not found_sheets_map:
            print("  No worksheets to check for columns.")
        else:
            for entity_name, original_sheet_name in found_sheets_map.items():
                required_columns = schema[entity_name]
                
                try:
                    sheet_df = pd.read_excel(target_xls, sheet_name=original_sheet_name)
                    actual_columns = sheet_df.columns.tolist()
                    missing_columns = [col for col in required_columns if col not in actual_columns]
                    
                    if missing_columns:
                        print(f"  [MISSING COLUMNS] In worksheet '{original_sheet_name}':")
                        print(f"    - The following required columns are missing: {missing_columns}")
                        validation_errors_this_file = True
                
                except Exception as e:
                    print(f"  [ERROR] Could not read or process worksheet '{original_sheet_name}'. Reason: {e}")
                    validation_errors_this_file = True
        
        #  5. Per-File Summary 
        print("\n--- Validation Complete for this file ---")
        if not validation_errors_this_file:
            print("Success! This file's structure matches the schema perfectly.")
        else:
            print("This file has one or more issues listed above.")

        #  6. Rename sheets, Fix IDs, and save the file while preserving formatting 
        print(f"\nUpdating worksheet names and fixing IDs in '{target_filename}' while preserving formatting...")
        try:
            # Load the workbook using openpyxl to preserve all styles and content
            workbook = openpyxl.load_workbook(target_filename)
            
            file_modified = False
            
            for sheet in workbook.worksheets:
                # --- A. Rename Sheet if needed ---
                if ' ' in sheet.title:
                    new_title = sheet.title.replace(' ', '_')
                    sheet.title = new_title
                    file_modified = True

                # --- B. Fix IDs in specific columns ---
                # 1. Identify columns that have "_ID" or "_ID_" in the header (Row 1)
                id_column_indices = []
                for cell in sheet[1]: # Iterate over the first row (headers)
                    if cell.value and isinstance(cell.value, str):
                        header_val = cell.value
                        if "_ID" in header_val or "_ID_" in header_val:
                            id_column_indices.append(cell.column) # Store 1-based column index

                # 2. Iterate through rows in identified columns to fix values
                if id_column_indices:
                    for col_idx in id_column_indices:
                        # Iterate from row 2 to the end
                        for row_idx in range(2, sheet.max_row + 1):
                            cell = sheet.cell(row=row_idx, column=col_idx)
                            val = cell.value
                            
                            if val and isinstance(val, str):
                                # Check if it starts with one of the prefixes (case insensitive)
                                lower_val = val.lower()
                                is_target_id = False
                                for prefix in id_prefixes:
                                    if lower_val.startswith(prefix.lower()):
                                        is_target_id = True
                                        break
                                
                                # If it matches a prefix and has double underscores, fix it
                                if is_target_id and "__" in val:
                                    # Replace 2 or more underscores with a single underscore
                                    new_val = re.sub(r'_{2,}', '_', val)
                                    cell.value = new_val
                                    file_modified = True
                                    # Optional: Print correction (commented out to keep output clean)
                                    # print(f"    Fixed ID in {sheet.title}: {val} -> {new_val}")

            if file_modified:
                # Save the workbook, overwriting the original file. This preserves formatting.
                workbook.save(target_filename)
                print(f"Successfully saved '{target_filename}' with updates.")
            else:
                print(f"No updates (renaming or ID fixing) needed for '{target_filename}'.")

        except Exception as e:
            print(f"  [ERROR] Could not save the updated file '{target_filename}'. Reason: {e}")

# Execute the validation and update function
validate_and_update_excel_structure()

Searching for files in: c:\Users\vojim\Desktop\ERA_IFF_FINAL\phase_1-excel_preparation_and_enrichment

Successfully loaded schema for 15 entities from 'columns_trace.xlsx'.
Found 2 target file(s) to validate and update.


 Inspecting File: BA-EG.xlsx 

Checking for missing worksheets...
  All required worksheets are present.

Checking for missing columns in existing worksheets...
  [MISSING COLUMNS] In worksheet 'EXPRESSION':
    - The following required columns are missing: ['expression_id', 'language_of_expression']
  [MISSING COLUMNS] In worksheet 'MANIFESTATION':
    - The following required columns are missing: ['manifestation_id', 'short_title_manifestation']
  [MISSING COLUMNS] In worksheet 'ITEM':
    - The following required columns are missing: ['item_id', 'material_item']
  [MISSING COLUMNS] In worksheet 'PLACE':
    - The following required columns are missing: ['place_id', 'description_wikidata_place']
  [MISSING COLUMNS] In worksheet 'PAGE':
    - The following required c